---
#### Bouncing Ball Animation from Leaning Scientific Programming with Python

* Direct link to the [work](https://scipython.com/book2/chapter-7-matplotlib/examples/animating-a-bouncing-ball/)
---

---

##### Design

* A function to generate bouncing ball height based on co-efficient of restitution

* Generate a pandas dataframe to store for 4 different values of restitution. Dataframe should have time, height and co-efficient of restitution. We can regnerate all ODE values like reaction kinetics, but keeping the use case simple here.

* User button option for restitution

* Time slider for bouncing ball

* Animate button 

* Add Tool tips to have height of ball and time

---

In [ ]:
import numpy as np
from bokeh.models import ColumnDataSource, Slider, Div, HoverTool, Grid, Tabs, Button

In [ ]:
# Acceleration due to gravity, m.s-2.
g = 9.81
# The maximum x-range of ball's trajectory to plot.
XMAX = 5
# The coefficient of restitution for bounces (-v_up/v_down).
cor_start = 0.65
# The time step for the animation.
dt = 0.005

In [ ]:
slider_cor = Slider(title="Co-efficient of Restitution"+" (initial: "+str(cor_start)+")", value=cor_start, start=0.1, end=1.0, step=0.05)

In [ ]:
start_time = 0.0
end_time = 8.0
time_step = 0.1
slider_time = Slider(title="Time Slider (s)", value=start_time, start=start_time, end=end_time, step=time_step, width=500)

In [ ]:
# Function to update the plot data
def update_data(attrname, old, new):
    cor = slider_cor.value
    new_data = dict(x=[], y=[])
    x, y, vx, vy = x0, y0, vx0, vy0
    while x < XMAX:
        x += vx0 * dt
        y += vy * dt
        vy -= g * dt
        if y < 0:
            # bounce!
            y = 0
            vy = -vy * cor
        new_data['x'].append(x)
        new_data['y'].append(y)
        source.data=new_data

In [ ]:
for w in [slider_cor, slider_time]:
    w.on_change('value', update_data)

In [ ]:
def animate():
    global callback_id
    if animate_button.label == '► Play':
        animate_button.label = '❚❚ Pause'
        callback_id = curdoc().add_periodic_callback(animate_update, time_step*1000.0) # s to milliseconds conversion
    else:
        animate_button.label = '► Play'
        curdoc().remove_periodic_callback(callback_id)

animate_button = Button(label='► Play', width=50)
animate_button.on_event('button_click', animate)

In [ ]:
# Setup layout

In [ ]:
# Setup server